In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:

!pip install pandas scikit-learn catboost

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from google.colab import files

from sklearn.ensemble import AdaBoostClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC

!pip install konlpy
from konlpy.tag import Okt

'''
MECAB 코드
!apt-get update
!apt-get install -y libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3 konlpy
from konlpy.tag import Mecab
from sklearn.feature_extraction.text import TfidfVectorizer
'''

df = pd.read_csv('/content/drive/MyDrive/데사/final_dataset.csv')

In [14]:

# 1) 레이블 추출
y = df['main_emotion']

# 2) 텍스트 토큰화 & TF–IDF
okt = Okt()
def tokenize(text):
    return okt.morphs(text)

tfidf = TfidfVectorizer(
    tokenizer=tokenize,
    token_pattern=None,
    max_features=500,
    ngram_range=(1,2)
)
X_text = tfidf.fit_transform(df['발화문'].fillna('')).toarray()

print("TF-IDF shape:", X_text.shape)

drop_cols = ['file_name', '발화문', 'main_emotion']
audio_cols = [c for c in df.columns if c not in drop_cols]
X_audio   = df[audio_cols].values

# 4) Early fusion
X = np.hstack([X_audio, X_text])

# 5) 레이블 인코딩
le    = LabelEncoder()
y_enc = le.fit_transform(y)

# 6) Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc, test_size=0.2, random_state=42, stratify=y_enc
)

# 7) 모델 학습 & 평가
def train_eval(name, model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"=== {name} Classification Report ===")
    print(classification_report(y_test, y_pred, target_names=le.classes_))

cb = CatBoostClassifier(verbose=0, random_state=42)
train_eval('CatBoost', cb)


TF-IDF shape: (13038, 500)
=== CatBoost Classification Report ===
              precision    recall  f1-score   support

       angry       0.46      0.42      0.44       434
     disgust       0.53      0.48      0.50       434
        fear       0.69      0.65      0.67       435
   happiness       0.71      0.74      0.73       435
     sadness       0.53      0.67      0.59       435
    surprise       0.69      0.64      0.66       435

    accuracy                           0.60      2608
   macro avg       0.60      0.60      0.60      2608
weighted avg       0.60      0.60      0.60      2608



In [8]:

# 2) 입력 X, 타깃 y 준비
y = df['main_emotion']

okt = Okt()

def tokenize(text):
    return okt.morphs(text)

# TF-IDF 적용 예시
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(
    tokenizer=tokenize,
    token_pattern=None,
    max_features=500,
    ngram_range=(1,2)
)
X_text = tfidf.fit_transform(df['발화문'].fillna('')).toarray()
print("TF-IDF shape:", X_text.shape)

'''
#mecab 코드
mecab = Mecab()

def tokenize(text):
    return mecab.morphs(text)

tfidf = TfidfVectorizer(
    tokenizer=tokenize,
    preprocessor=lambda x: x,   # 전처리 비활성화
    token_pattern=None,         # token_pattern 은 무시
    max_features=500,
    ngram_range=(1,2)
)
X_text = tfidf.fit_transform(df['발화문'].fillna('')).toarray()
'''

# 4) 오디오 피처: 수치 컬럼만 선택
audio_cols = [c for c in df.columns if c not in ['file_name', '발화문', 'main_emotion']]
X_audio   = df[audio_cols].values

# 5) Early fusion: concatenate
X = np.hstack([X_audio, X_text])

# 6) 레이블 인코딩
le = LabelEncoder()
y_enc = le.fit_transform(y)

# 7) Train/Test 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc, test_size=0.2, random_state=42, stratify=y_enc
)

# 8) 모델 정의 & 학습/평가 함수
def train_eval(name, model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"=== {name} Classification Report ===")
    print(classification_report(y_test, y_pred, target_names=le.classes_))


# CatBoost
cb = CatBoostClassifier(verbose=0, random_state=42)
train_eval('CatBoost', cb)



TF-IDF shape: (13038, 500)


CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=0]="5f98491ef8fac448cc0a8463": Cannot convert '5f98491ef8fac448cc0a8463' to float